# **Bitcoin price prediction - Block Split**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: perform model's train / validation with hyperparameter tuning and cross validation based on different methods of splitting the dataset.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = False # Decide whether or not to use operations that might slow down notebook execution
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

## Import my utilities

In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

# Import my utilities
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
from config import *
import train_validation_utilities

importlib.reload(train_validation_utilities)

<module 'train_validation_utilities' from 'D:\\Documents/Repository/BDC/project/utilities\\train_validation_utilities.py'>

## Core variables

In [4]:
# BS = Block Split
# WFS = Walk Forward Split
# SS = Single Split
SPLITTING_METHOD = BS

# LR = LinearRegression 
# GLR = GeneralizedLinearRegression 
# RF = RandomForestRegressor 
# GBTR = GradientBoostingTreeRegressor
MODEL_NAME = LR

In [5]:
###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features paths
FEATURES_CORRELATION = FEATURES_DIR + "/" + FEATURES_CORRELATION_LABEL + ".json"
BASE_FEATURES = FEATURES_DIR + "/" + BASE_FEATURES_LABEL + ".json"
BASE_AND_MOST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_MOST_CORR_FEATURES_LABEL + ".json"
BASE_AND_LEAST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_LEAST_CORR_FEATURES_LABEL + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
ALL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_all.csv"
REL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_rel.csv"

MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [6]:
# Importing useful libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pio.renderers.default = 'vscode+colab' # To correctly render plotly plots

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
train_validation_utilities.dataset_info(df)

+-------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+--------------+--------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|          timestamp| id|     market-price|    opening-price|    highest-price|     lowest-price|    closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|n-transactions|estimated-transaction-volume-usd|       sma-5-days|   

# Loading features

In [10]:
# Loading base features
with open(BASE_FEATURES, "r") as f:
    BASE_FEATURES = json.load(f)
print(BASE_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd']


In [11]:
# Loading currency and additional most correlated features
with open(BASE_AND_MOST_CORR_FEATURES, "r") as f:
    BASE_AND_MOST_CORR_FEATURES = json.load(f)
print(BASE_AND_MOST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days']


In [12]:
# Loading currency and additional least correlated features
with open(BASE_AND_LEAST_CORR_FEATURES, "r") as f:
    BASE_AND_LEAST_CORR_FEATURES = json.load(f)
print(BASE_AND_LEAST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'sma-100-days', 'transaction-fees-usd', 'n-unique-addresses', 'sma-50-days', 'n-transactions-total', 'blocks-size', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- `Default without normalization:` make predictions using the base model
- `Default with normalization:` like the previous one but features are normalized

Then the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:
- `Hyperparameter tuning:` finding the best parameters to use. 
- `Cross Validation:` validate the performance of the model with the chosen parameters (also here using Block split / Walk forward split)

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved in order to make predictions on the test set.

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem). In this case the `Block time series splits` method will be used: involves dividing the time series into blocks of equal length, and then using each block as a separate fold for cross-validation.

<img src="https://github.com/CorsiDanilo/big-data-computing-project/blob/main/notebooks/images/block-splits.png?raw=1">

In [13]:
# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'block_splits', 'splits': 5}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

In [14]:
# Get default parameters
params = train_validation_utilities.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [100], 'regParam': [0.0], 'elasticNetParam': [0.0]}

### Without normalization

In [15]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [16]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [17]:
# Make predictions by using base features
default_train_results_base_features, default_valid_results_base_features, default_train_pred_base_features, default_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


In [18]:
default_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,train,block_splits,base_features,1,"(20981, 5246)","[100, 0.0, 0.0]",2.411945,5.817477,1.744119,0.000217,0.999996,0.999996,1.510148
1,LinearRegression,default,train,block_splits,base_features,2,"(20981, 5246)","[100, 0.0, 0.0]",10.900370,118.818077,6.445919,0.000221,1.000000,1.000000,0.429968
2,LinearRegression,default,train,block_splits,base_features,3,"(20981, 5246)","[100, 0.0, 0.0]",14.122589,199.447530,10.690630,0.000237,0.999998,0.999998,0.379714
3,LinearRegression,default,train,block_splits,base_features,4,"(20981, 5246)","[100, 0.0, 0.0]",9.353565,87.489169,6.363338,0.000211,0.999999,0.999999,0.413036
4,LinearRegression,default,train,block_splits,base_features,5,"(20981, 5246)","[100, 0.0, 0.0]",4.659931,21.714956,3.194134,0.000133,0.999999,0.999999,0.415337


In [19]:
default_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,valid,block_splits,base_features,1,"(20981, 5246)","[100, 0.0, 0.0]",2.182279,4.762341,1.463723,0.000141,0.999996,0.999996,1.510148
1,LinearRegression,default,valid,block_splits,base_features,2,"(20981, 5246)","[100, 0.0, 0.0]",19.064293,363.447269,13.730028,0.000247,0.999967,0.999967,0.429968
2,LinearRegression,default,valid,block_splits,base_features,3,"(20981, 5246)","[100, 0.0, 0.0]",10.713960,114.788945,8.520846,0.000202,0.999994,0.999994,0.379714
3,LinearRegression,default,valid,block_splits,base_features,4,"(20981, 5246)","[100, 0.0, 0.0]",5.359726,28.726667,3.211948,0.000170,0.999967,0.999967,0.413036
4,LinearRegression,default,valid,block_splits,base_features,5,"(20981, 5246)","[100, 0.0, 0.0]",3.746327,14.034967,2.751310,0.000092,0.999973,0.999973,0.415337


In [20]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [21]:
# Make predictions by using base and most additional correlated features
default_train_results_base_and_most_corr_features, default_valid_results_base_and_most_corr_features, default_train_pred_base_and_most_corr_features, default_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


In [22]:
default_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,train,block_splits,base_and_most_corr_features,1,"(20981, 5246)","[100, 0.0, 0.0]",2.376878,5.649548,1.746132,0.000217,0.999996,0.999996,0.438629
1,LinearRegression,default,train,block_splits,base_and_most_corr_features,2,"(20981, 5246)","[100, 0.0, 0.0]",10.716564,114.844747,6.443820,0.000223,1.000000,1.000000,0.473491
2,LinearRegression,default,train,block_splits,base_and_most_corr_features,3,"(20981, 5246)","[100, 0.0, 0.0]",13.854656,191.951483,10.618452,0.000236,0.999998,0.999998,0.414090
3,LinearRegression,default,train,block_splits,base_and_most_corr_features,4,"(20981, 5246)","[100, 0.0, 0.0]",9.288450,86.275295,6.363632,0.000211,0.999999,0.999999,0.442272
4,LinearRegression,default,train,block_splits,base_and_most_corr_features,5,"(20981, 5246)","[100, 0.0, 0.0]",4.628985,21.427502,3.208231,0.000134,0.999999,0.999999,0.414950


In [23]:
default_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,valid,block_splits,base_and_most_corr_features,1,"(20981, 5246)","[100, 0.0, 0.0]",5.657399,32.006160,4.638052,0.000444,0.999971,0.999971,0.438629
1,LinearRegression,default,valid,block_splits,base_and_most_corr_features,2,"(20981, 5246)","[100, 0.0, 0.0]",18.566341,344.709035,13.484018,0.000243,0.999968,0.999968,0.473491
2,LinearRegression,default,valid,block_splits,base_and_most_corr_features,3,"(20981, 5246)","[100, 0.0, 0.0]",45.861604,2103.286740,39.718431,0.000969,0.999885,0.999885,0.414090
3,LinearRegression,default,valid,block_splits,base_and_most_corr_features,4,"(20981, 5246)","[100, 0.0, 0.0]",13.380395,179.034963,10.521873,0.000543,0.999794,0.999793,0.442272
4,LinearRegression,default,valid,block_splits,base_and_most_corr_features,5,"(20981, 5246)","[100, 0.0, 0.0]",5.479394,30.023762,4.656504,0.000156,0.999942,0.999942,0.414950


In [24]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [25]:
# Make predictions by using base and least additional correlated features
default_train_results_base_and_least_corr_features, default_valid_results_base_and_least_corr_features, default_train_pred_base_and_least_corr_features, default_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:01<00:00,  1.74s/it]


In [26]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,train,block_splits,base_and_least_corr_features,1,"(20981, 5246)","[100, 0.0, 0.0]",4.187459,17.534812,3.030993,0.000386,0.999989,0.999989,1.742847
1,LinearRegression,default,train,block_splits,base_and_least_corr_features,2,"(20981, 5246)","[100, 0.0, 0.0]",10.684763,114.164150,6.416597,0.000221,1.000000,1.000000,0.444876
2,LinearRegression,default,train,block_splits,base_and_least_corr_features,3,"(20981, 5246)","[100, 0.0, 0.0]",13.784478,190.011833,10.518721,0.000234,0.999998,0.999998,0.484525
3,LinearRegression,default,train,block_splits,base_and_least_corr_features,4,"(20981, 5246)","[100, 0.0, 0.0]",9.253749,85.631869,6.356062,0.000211,0.999999,0.999999,0.455146
4,LinearRegression,default,train,block_splits,base_and_least_corr_features,5,"(20981, 5246)","[100, 0.0, 0.0]",4.586932,21.039948,3.204851,0.000134,0.999999,0.999999,0.502468


In [27]:
default_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,valid,block_splits,base_and_least_corr_features,1,"(20981, 5246)","[100, 0.0, 0.0]",5.067433,25.678874,4.122174,0.000401,0.999977,0.999977,1.742847
1,LinearRegression,default,valid,block_splits,base_and_least_corr_features,2,"(20981, 5246)","[100, 0.0, 0.0]",18.626399,346.942756,13.874361,0.000250,0.999968,0.999968,0.444876
2,LinearRegression,default,valid,block_splits,base_and_least_corr_features,3,"(20981, 5246)","[100, 0.0, 0.0]",19.935985,397.443510,17.173459,0.000414,0.999978,0.999978,0.484525
3,LinearRegression,default,valid,block_splits,base_and_least_corr_features,4,"(20981, 5246)","[100, 0.0, 0.0]",5.673738,32.191308,3.694466,0.000195,0.999963,0.999963,0.455146
4,LinearRegression,default,valid,block_splits,base_and_least_corr_features,5,"(20981, 5246)","[100, 0.0, 0.0]",8.311389,69.079192,7.354502,0.000246,0.999867,0.999867,0.502468


### With normalization

In [28]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [29]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [30]:
# Make predictions by using base features
default_norm_train_results_base_features, default_norm_valid_results_base_features, default_norm_train_pred_base_features, default_norm_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:01<00:00,  1.52s/it]


In [31]:
default_norm_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,train,block_splits,base_features_norm,1,"(20981, 5246)","[100, 0.0, 0.0]",169.714760,2.880310e+04,124.530628,0.015842,0.981330,0.981326,0.381080
1,LinearRegression,default_norm,train,block_splits,base_features_norm,2,"(20981, 5246)","[100, 0.0, 0.0]",5294.110657,2.802761e+07,4364.636224,0.211993,0.882980,0.882958,0.407647
2,LinearRegression,default_norm,train,block_splits,base_features_norm,3,"(20981, 5246)","[100, 0.0, 0.0]",1924.741992,3.704632e+06,1662.521089,0.036512,0.962928,0.962921,0.399886
3,LinearRegression,default_norm,train,block_splits,base_features_norm,4,"(20981, 5246)","[100, 0.0, 0.0]",1700.152352,2.890518e+06,1387.187225,0.050076,0.965728,0.965722,0.356136
4,LinearRegression,default_norm,train,block_splits,base_features_norm,5,"(20981, 5246)","[100, 0.0, 0.0]",632.832551,4.004770e+05,515.603693,0.022671,0.981246,0.981243,0.375814


In [32]:
default_norm_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,valid,block_splits,base_features_norm,1,"(20981, 5246)","[100, 0.0, 0.0]",532.301266,2.833446e+05,350.622459,0.031436,0.743715,0.743519,0.381080
1,LinearRegression,default_norm,valid,block_splits,base_features_norm,2,"(20981, 5246)","[100, 0.0, 0.0]",10599.395302,1.123472e+08,10155.943090,0.178932,-9.340545,-9.348437,0.407647
2,LinearRegression,default_norm,valid,block_splits,base_features_norm,3,"(20981, 5246)","[100, 0.0, 0.0]",1642.038546,2.696291e+06,1524.198740,0.034902,0.852341,0.852229,0.399886
3,LinearRegression,default_norm,valid,block_splits,base_features_norm,4,"(20981, 5246)","[100, 0.0, 0.0]",1919.092022,3.682914e+06,1402.151722,0.074705,-3.245964,-3.249205,0.356136
4,LinearRegression,default_norm,valid,block_splits,base_features_norm,5,"(20981, 5246)","[100, 0.0, 0.0]",1673.348467,2.800095e+06,1527.019641,0.050798,-4.373631,-4.377732,0.375814


In [33]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [34]:
# Make predictions by using base and most additional correlated features
default_norm_train_results_base_and_most_corr_features, default_norm_valid_results_base_and_most_corr_features, default_norm_train_pred_base_and_most_corr_features, default_norm_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


In [35]:
default_norm_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,train,block_splits,base_and_most_corr_features_norm,1,"(20981, 5246)","[100, 0.0, 0.0]",141.973662,2.015652e+04,107.123955,0.013545,0.986934,0.986932,0.409561
1,LinearRegression,default_norm,train,block_splits,base_and_most_corr_features_norm,2,"(20981, 5246)","[100, 0.0, 0.0]",3148.874214,9.915409e+06,2638.766975,0.139249,0.958602,0.958594,0.392990
2,LinearRegression,default_norm,train,block_splits,base_and_most_corr_features_norm,3,"(20981, 5246)","[100, 0.0, 0.0]",1224.130961,1.498497e+06,996.854776,0.022609,0.985005,0.985002,0.425319
3,LinearRegression,default_norm,train,block_splits,base_and_most_corr_features_norm,4,"(20981, 5246)","[100, 0.0, 0.0]",1363.225917,1.858385e+06,1110.797756,0.037492,0.977966,0.977962,0.454231
4,LinearRegression,default_norm,train,block_splits,base_and_most_corr_features_norm,5,"(20981, 5246)","[100, 0.0, 0.0]",432.797696,1.873138e+05,355.235875,0.016266,0.991228,0.991227,0.424075


In [36]:
default_norm_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,valid,block_splits,base_and_most_corr_features_norm,1,"(20981, 5246)","[100, 0.0, 0.0]",372.710436,1.389131e+05,340.892267,0.033297,0.874353,0.874257,0.409561
1,LinearRegression,default_norm,valid,block_splits,base_and_most_corr_features_norm,2,"(20981, 5246)","[100, 0.0, 0.0]",5714.787403,3.265880e+07,4846.219334,0.087936,-2.005948,-2.008242,0.392990
2,LinearRegression,default_norm,valid,block_splits,base_and_most_corr_features_norm,3,"(20981, 5246)","[100, 0.0, 0.0]",2766.193820,7.651828e+06,2259.782462,0.055155,0.580958,0.580638,0.425319
3,LinearRegression,default_norm,valid,block_splits,base_and_most_corr_features_norm,4,"(20981, 5246)","[100, 0.0, 0.0]",2488.034891,6.190318e+06,1990.781191,0.100003,-6.136704,-6.142151,0.454231
4,LinearRegression,default_norm,valid,block_splits,base_and_most_corr_features_norm,5,"(20981, 5246)","[100, 0.0, 0.0]",2103.977438,4.426721e+06,2035.331898,0.067907,-7.495270,-7.501754,0.424075


In [37]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [38]:
# Make predictions by using base and least additional correlated features
default_norm_train_results_base_and_least_corr_features, default_norm_valid_results_base_and_least_corr_features, default_norm_train_pred_base_and_least_corr_features, default_norm_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


In [39]:
default_norm_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,train,block_splits,base_and_least_corr_features_norm,1,"(20981, 5246)","[100, 0.0, 0.0]",47.080981,2216.618734,36.972585,0.004579,0.998563,0.998563,0.399794
1,LinearRegression,default_norm,train,block_splits,base_and_least_corr_features_norm,2,"(20981, 5246)","[100, 0.0, 0.0]",304.510852,92726.859086,236.588043,0.010965,0.999613,0.999613,0.603015
2,LinearRegression,default_norm,train,block_splits,base_and_least_corr_features_norm,3,"(20981, 5246)","[100, 0.0, 0.0]",848.154077,719365.338396,668.080783,0.014803,0.992801,0.992800,0.467111
3,LinearRegression,default_norm,train,block_splits,base_and_least_corr_features_norm,4,"(20981, 5246)","[100, 0.0, 0.0]",148.934568,22181.505680,117.360821,0.003915,0.999737,0.999737,0.553107
4,LinearRegression,default_norm,train,block_splits,base_and_least_corr_features_norm,5,"(20981, 5246)","[100, 0.0, 0.0]",142.755192,20379.044898,112.002463,0.004853,0.999046,0.999046,0.469985


In [40]:
default_norm_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,valid,block_splits,base_and_least_corr_features_norm,1,"(20981, 5246)","[100, 0.0, 0.0]",287.980471,8.293275e+04,204.474134,0.018631,0.924987,0.924930,0.399794
1,LinearRegression,default_norm,valid,block_splits,base_and_least_corr_features_norm,2,"(20981, 5246)","[100, 0.0, 0.0]",2662.826320,7.090644e+06,1834.917532,0.032348,0.347370,0.346872,0.603015
2,LinearRegression,default_norm,valid,block_splits,base_and_least_corr_features_norm,3,"(20981, 5246)","[100, 0.0, 0.0]",5279.734146,2.787559e+07,4580.734391,0.110974,-0.526570,-0.527735,0.467111
3,LinearRegression,default_norm,valid,block_splits,base_and_least_corr_features_norm,4,"(20981, 5246)","[100, 0.0, 0.0]",563.542103,3.175797e+05,477.179211,0.024807,0.633868,0.633589,0.553107
4,LinearRegression,default_norm,valid,block_splits,base_and_least_corr_features_norm,5,"(20981, 5246)","[100, 0.0, 0.0]",2342.978351,5.489548e+06,2254.424348,0.075642,-9.534928,-9.542969,0.469985


In [41]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Dataset', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [42]:
# Define the results to show
default_comparison_lst = [default_valid_results_base_features, default_valid_results_base_and_most_corr_features, default_valid_results_base_and_least_corr_features, default_norm_valid_results_base_features, default_norm_valid_results_base_and_most_corr_features, default_norm_valid_results_base_and_least_corr_features]

# Show the comparison table
default_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,valid,block_splits,base_features,"[100, 0.0, 0.0]",8.213317,1.051520e+02,5.935571,0.000170,0.999979,0.999979,0.629641
0,LinearRegression,default,valid,block_splits,base_and_most_corr_features,"[100, 0.0, 0.0]",17.789027,5.378121e+02,14.603776,0.000471,0.999912,0.999912,0.436686
0,LinearRegression,default,valid,block_splits,base_and_least_corr_features,"[100, 0.0, 0.0]",11.522989,1.742671e+02,9.243792,0.000301,0.999951,0.999951,0.725972
0,LinearRegression,default_norm,valid,block_splits,base_features_norm,"[100, 0.0, 0.0]",3273.235120,2.436197e+07,2991.987130,0.074154,-3.072817,-3.075925,0.384113
0,LinearRegression,default_norm,valid,block_splits,base_and_most_corr_features_norm,"[100, 0.0, 0.0]",2689.140798,1.021332e+07,2294.601430,0.068860,-2.836522,-2.839450,0.421235
0,LinearRegression,default_norm,valid,block_splits,base_and_least_corr_features_norm,"[100, 0.0, 0.0]",2227.412278,8.171259e+06,1870.345923,0.052480,-1.631054,-1.633063,0.498602


In [43]:
# Save the best default model results and predicitons
best_default_results = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in [default_norm_valid_results_base_and_least_corr_features]])
best_default_predictions = default_norm_valid_pred_base_and_least_corr_features
best_default_results

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,valid,block_splits,base_and_least_corr_features_norm,"[100, 0.0, 0.0]",2227.412278,8.171259e+06,1870.345923,0.05248,-1.631054,-1.633063,0.498602


In [44]:
# Define the choosen features
FEATURES_NORMALIZATION = True
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

In [ ]:
# Get model grid parameters
params = train_validation_utilities.get_model_grid_params(MODEL_NAME)
params

### Hyperparameter tuning

In [ ]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"

In [ ]:
# Perform hyperparameter tuning
hyp_res = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)
hyp_res

---
Since during this stage will be used the Block split / Walk forward split method of the dataset I compute a score for each parameter chosen by each split, assigning weights based on:
   * Their `frequency` for each split (if the same parameters are chosen from several splits, these will have greater weight) 
   * The `split` they belong to (the closer the split is to today's date the more weight they will have)
   * Their `RMSE value` for each split (the lower this is, the more weight they will have)
   
   Then, the overall score will be calculated by putting together these three weights for each parameter and the one with the best score will be the chosen parameter.

In [ ]:
# Show parameters score
grouped_scores, best_params = train_validation_utilities.choose_best_params(hyp_res)
grouped_scores

In [ ]:
# Print best parameters
print(f"Best parameters: {best_params}")

### Cross validation

In [ ]:
MODEL_TYPE = "cross_val"

In [ ]:
# Get tuned parameters
params = train_validation_utilities.get_best_model_params(best_params, MODEL_NAME)
params

In [ ]:
# Perform cross validation
cv_train_result, cv_valid_result, cv_train_pred, cv_valid_pred = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [ ]:
cv_train_result

In [ ]:
cv_valid_result

In [ ]:
# Define the results to show
tuned_comparison_lst = [cv_valid_result]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

# Comparison table
Visualization of model performance at various stages of train / validation

In [ ]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df], ignore_index=True))
final_comparison_lst_df

# Model accuracy

Since predicting the price accurately is very difficult I also saw how good the models are at predicting whether the price will go up or down in this way:

For each prediction let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
I compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that I count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally I compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [ ]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(cv_valid_pred)

# Compute model accuracy
default_accuracy = train_validation_utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = train_validation_utilities.model_accuracy(validated_pred_spark)

# Shows whether features are normalised or not
if FEATURES_NORMALIZATION:
    NEW_CHOSEN_FEATURES_LABEL = CHOSEN_FEATURES_LABEL + "_norm"
    CHOSEN_FEATURES_LABEL = NEW_CHOSEN_FEATURES_LABEL
    
# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (tuned)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (tuned): {validated_accuracy:.2f}%")

In [ ]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, cv_valid_result]
default_tuned_results_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

# Saving final results

In [ ]:
# Save all final comparison results
final_comparison_lst_df.to_csv(ALL_MODEL_RESULTS, index=False)

In [ ]:
# Save relevant results (default and tuned results)
default_tuned_results_df.to_csv(REL_MODEL_RESULTS, index=False)

In [ ]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)

In [ ]:
# Export notebook in html format (remember to save the notebook and change the model name)
if LOCAL_RUNNING:
    !jupyter nbconvert --to html 3-block-split_{MODEL_NAME}.ipynb --output 3-block-split_{MODEL_NAME} --output-dir='./exports'